<a href="https://colab.research.google.com/github/gitHubAndyLee2020/Transformer_Seq2Seq_Ko_En_Bidirectional_Translator/blob/main/korean_english_bydirectional_translator_project_v5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Korean-English Bidirectional Translator

### Bidirectional Model + Performance Tests

### Data Source

- https://www.kaggle.com/datasets/msarmi9/englishkorean-multitarget-ted-talks-task-mttt

### Upload Data

- Download `kr_en_translation_ted_talk_dataset.zip` from the GitHub repo, and upload it using the code below.

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving kr_en_translation_ted_talk_dataset.zip to kr_en_translation_ted_talk_dataset.zip


In [ ]:
!unzip kr_en_translation_ted_talk_dataset.zip

Archive:  kr_en_translation_ted_talk_dataset.zip
  inflating: multitarget-ted/en-ko/raw/ted_dev_en-ko.raw.en  
  inflating: multitarget-ted/en-ko/raw/ted_dev_en-ko.raw.ko  
  inflating: multitarget-ted/en-ko/raw/ted_test1_en-ko.raw.en  
  inflating: multitarget-ted/en-ko/raw/ted_test1_en-ko.raw.ko  
  inflating: multitarget-ted/en-ko/raw/ted_train_en-ko.raw.en  
  inflating: multitarget-ted/en-ko/raw/ted_train_en-ko.raw.ko  


In [ ]:
import os

# Path to the folder containing the text files
folder_path = 'multitarget-ted/en-ko/raw/'

# List all files in the folder
files = os.listdir(folder_path)

# Loop through each file and count the number of lines
for file_name in files:
    with open(os.path.join(folder_path, file_name), 'r', encoding='utf-8') as f:
        lines = f.readlines()
        num_lines = len(lines)
        print(f"{file_name} has {num_lines} lines.")

ted_dev_en-ko.raw.ko has 1958 lines.
ted_test1_en-ko.raw.ko has 1982 lines.
ted_dev_en-ko.raw.en has 1958 lines.
ted_test1_en-ko.raw.en has 1982 lines.
ted_train_en-ko.raw.en has 166215 lines.
ted_train_en-ko.raw.ko has 166215 lines.


### Prepare Data (Korean-to-English)

In [ ]:
FILE_PATH = 'multitarget-ted/en-ko/raw/'

In [ ]:
!pip install 'portalocker>=2.0.0'

In [ ]:
!pip install -U torchdata
!pip install -U spacy
!python3 -m spacy download en_core_web_sm
!python3 -m spacy download ko_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 6.6 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.6.1
    Uninstalling spacy-3.6.1:
      Successfully uninstalled spacy-3.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.6.0 requires spacy<3.7.0,>=3.6.0, but you have spacy 3.7.2 which is incompatible.
2023-10-22 22:42:20.494775: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-22 22:42:20.550286: I tensorflow/core

In [ ]:
from typing import Iterable, List
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
import math
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from timeit import default_timer as timer
import os

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Create custom data iterator
def custom_data_iterator(SRC_LANGUAGE, TGT_LANGUAGE, file_path_src, file_path_tgt):
    with open(file_path_src, 'r', encoding='utf-8') as f_src, \
         open(file_path_tgt, 'r', encoding='utf-8') as f_tgt:
        for src_sentence, tgt_sentence in zip(f_src, f_tgt):
            yield {SRC_LANGUAGE: src_sentence.strip(), TGT_LANGUAGE: tgt_sentence.strip()}

In [ ]:
# Helper function to yield list of tokens
def yield_tokens(token_transform, data_iter: Iterable, language: str) -> List[str]:
    for data_sample in data_iter:
        yield token_transform[language](data_sample[language])

In [ ]:
# Define special symbols and indices
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

In [ ]:
# Build vocab from source and target language datasets
def build_vocab_and_transforms(SRC_LANGUAGE, TGT_LANGUAGE, train_data_src, train_data_tgt):
    token_transform = {}
    token_transform[SRC_LANGUAGE] = get_tokenizer('spacy', language='ko_core_news_sm' if SRC_LANGUAGE == 'ko' else 'en_core_web_sm')
    token_transform[TGT_LANGUAGE] = get_tokenizer('spacy', language='en_core_web_sm' if TGT_LANGUAGE == 'en' else 'ko_core_news_sm')

    vocab_transform = {}
    for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
        train_iter = custom_data_iterator(SRC_LANGUAGE, TGT_LANGUAGE, train_data_src, train_data_tgt)
        vocab_transform[ln] = build_vocab_from_iterator(yield_tokens(token_transform, train_iter, ln),
                                                        min_freq=1,
                                                        specials=special_symbols,
                                                        special_first=True)
        vocab_transform[ln].set_default_index(UNK_IDX)
    return vocab_transform, token_transform

In [ ]:
# helper Module that adds positional encoding to the token embedding to introduce a notion of word order.
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# Seq2Seq Network
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

In [ ]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [ ]:
torch.manual_seed(0)

EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
BATCH_SIZE = 8 # reduced further to allow for more training epochs
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3

In [ ]:
# helper function to club together sequential operations
def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

# function to add BOS/EOS and create tensor for input sequence indices
def tensor_transform(token_ids: List[int]):
    return torch.cat((torch.tensor([BOS_IDX]),
                      torch.tensor(token_ids),
                      torch.tensor([EOS_IDX])))

# function to transform string data to tensor
def string_to_tensor_transform(SRC_LANGUAGE, TGT_LANGUAGE, token_transform, vocab_transform):
    # ``src`` and ``tgt`` language text transforms to convert raw strings into tensors indices
    text_transform = {}
    for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
        text_transform[ln] = sequential_transforms(token_transform[ln], #Tokenization
                                                  vocab_transform[ln], #Numericalization
                                                  tensor_transform) # Add BOS/EOS and create tensor
    return text_transform

In [ ]:
def count_parameters(model, verbose=False):
    total_params = 0
    for name, param in model.named_parameters():
        num_params = param.numel()
        if verbose:
          print(f"Layer {name} has {num_params} parameters")
        total_params += num_params
    print(f"Total number of parameters: {total_params}")

In [ ]:
def init_transformer_model(SRC_LANGUAGE, TGT_LANGUAGE, train_data_src, train_data_tgt, val_data_src, val_data_tgt):
    print("====================================================================================================")
    print(f"Initializing model for {SRC_LANGUAGE}-to-{TGT_LANGUAGE} translation...")

    vocab_transform, token_transform = build_vocab_and_transforms(SRC_LANGUAGE, TGT_LANGUAGE, train_data_src, train_data_tgt)
    text_transform = string_to_tensor_transform(SRC_LANGUAGE, TGT_LANGUAGE, token_transform, vocab_transform)

    SRC_VOCAB_SIZE = len(vocab_transform[SRC_LANGUAGE])
    TGT_VOCAB_SIZE = len(vocab_transform[TGT_LANGUAGE])

    # Output the vocabulary size for each source and target languages
    print(f"Source Vocabulary Size: {SRC_VOCAB_SIZE}")
    print(f"Target Vocabulary Size: {TGT_VOCAB_SIZE}")

    transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                    NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)

    for p in transformer.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)

    transformer = transformer.to(DEVICE)

    loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

    optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

    # Output the parameter number of the model
    count_parameters(transformer, verbose=False)

    print("====================================================================================================\n\n")

    return transformer, optimizer, loss_fn, vocab_transform, text_transform

In [ ]:
# function to collate data samples into batch tensors
def collate_fn(batch, SRC_LANGUAGE, TGT_LANGUAGE, text_transform):
    src_batch, tgt_batch = [], []
    for sample in batch:
        src_sample = sample[SRC_LANGUAGE]
        tgt_sample = sample[TGT_LANGUAGE]
        src_batch.append(text_transform[SRC_LANGUAGE](src_sample.rstrip("\n")))
        tgt_batch.append(text_transform[TGT_LANGUAGE](tgt_sample.rstrip("\n")))

    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch, tgt_batch

In [ ]:
# Assuming custom_data_iterator is the function to read your data from text files
# It should yield dictionaries with SRC_LANGUAGE and TGT_LANGUAGE as keys
# E.g., {'ko': "안녕하세요", 'en': "Hello"}

def train_epoch(model, optimizer, loss_fn, SRC_LANGUAGE, TGT_LANGUAGE, train_data_src, train_data_tgt, text_transform):
    model.train()
    losses = 0

    # Replace Multi30k with your custom data iterator
    train_iter = custom_data_iterator(SRC_LANGUAGE, TGT_LANGUAGE, train_data_src, train_data_tgt)
    train_dataloader = DataLoader(list(train_iter), batch_size=BATCH_SIZE, collate_fn=lambda b: collate_fn(b, SRC_LANGUAGE, TGT_LANGUAGE, text_transform))

    for batch in train_dataloader:
        src, tgt = batch[0].to(DEVICE), batch[1].to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask, src_padding_mask)

        optimizer.zero_grad()

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()

    return losses / len(train_dataloader)


def evaluate(model, loss_fn, SRC_LANGUAGE, TGT_LANGUAGE, dev_data_src, dev_data_tgt, text_transform):
    model.eval()
    losses = 0

    # Replace Multi30k with your custom data iterator
    val_iter = custom_data_iterator(SRC_LANGUAGE, TGT_LANGUAGE, dev_data_src, dev_data_tgt)
    val_dataloader = DataLoader(list(val_iter), batch_size=BATCH_SIZE, collate_fn=lambda b: collate_fn(b, SRC_LANGUAGE, TGT_LANGUAGE, text_transform))

    for batch in val_dataloader:
        src, tgt = batch[0].to(DEVICE), batch[1].to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask, src_padding_mask)

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        losses += loss.item()

    return losses / len(val_dataloader)

In [ ]:
def save_checkpoint(model, filename):
    torch.save(model.state_dict(), filename)
    print(f"Model saved to {filename}")

def load_checkpoint(model, filename):
    model.load_state_dict(torch.load(filename))
    print(f"Model loaded from {filename}")

In [ ]:
def train_model(model, optimizer, loss_fn, SRC_LANGUAGE, TGT_LANGUAGE, train_data_src, train_data_tgt, dev_data_src, dev_data_tgt, text_transform, NUM_EPOCHS):
  print(f"Training {SRC_LANGUAGE}-to-{TGT_LANGUAGE} translator model...")
  total_start_time = timer()
  for epoch in range(1, NUM_EPOCHS+1):
      start_time = timer()
      train_loss = train_epoch(model, optimizer, loss_fn, SRC_LANGUAGE, TGT_LANGUAGE, train_data_src, train_data_tgt, text_transform)
      end_time = timer()
      val_loss = evaluate(model, loss_fn, SRC_LANGUAGE, TGT_LANGUAGE, dev_data_src, dev_data_tgt, text_transform)

      print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))

      # Check if checkpoints folder exists, if not create it
      if not os.path.exists("checkpoints"):
          os.makedirs("checkpoints")

      save_checkpoint(model, f"checkpoints/translator_{SRC_LANGUAGE}_to_{TGT_LANGUAGE}_epoch_{epoch}.pth")
  total_end_time = timer()
  print(f"Total training time = {(total_end_time - total_start_time):.3f}s")

# function to generate output sequence using greedy algorithm
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                    .type(torch.bool)).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
            break
    return ys


# actual function to translate input sentence into target language
def translate(model: torch.nn.Module, src_sentence: str, SRC_LANGUAGE, TGT_LANGUAGE, vocab_transform, text_transform):
    model.eval()
    src = text_transform[SRC_LANGUAGE](src_sentence).view(-1, 1)
    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(
        model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
    return " ".join(vocab_transform[TGT_LANGUAGE].lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")

In [ ]:
def identify_language(src_sentence: str):
    for char in src_sentence:
        # Check if character is a Korean character
        if ord('가') <= ord(char) <= ord('힣'):
            return 'ko'
    # If the loop completes, then no Korean characters were found
    return 'en'

In [ ]:
class BidirectionalTranslator(nn.Module):
    def __init__(self, transformer_ko_to_en, vocab_transform_ko_to_en, text_transform_ko_to_en, transformer_en_to_ko, vocab_transform_en_to_ko, text_transform_en_to_ko):
        super(BidirectionalTranslator, self).__init__()
        self.transformer_ko_to_en = transformer_ko_to_en
        self.vocab_transform_ko_to_en = vocab_transform_ko_to_en
        self.text_transform_ko_to_en = text_transform_ko_to_en
        self.transformer_en_to_ko = transformer_en_to_ko
        self.vocab_transform_en_to_ko = vocab_transform_en_to_ko
        self.text_transform_en_to_ko = text_transform_en_to_ko

    def forward(self, src_sentence: str):  # Note that it's "forward", not "call"
        ln = identify_language(src_sentence)
        if ln == 'ko':
            return translate(self.transformer_ko_to_en, src_sentence, 'ko', 'en', self.vocab_transform_ko_to_en, self.text_transform_ko_to_en)
        elif ln == 'en':
            return translate(self.transformer_en_to_ko, src_sentence, 'en', 'ko', self.vocab_transform_en_to_ko, self.text_transform_en_to_ko)
        return 'Invalid language; please input either Korean or English.'

In [ ]:
def run_test(translator, epoch_index, test_data_ko, test_data_en):
    start_time = timer()
    # Create directory if it doesn't exist
    if not os.path.exists("translated"):
        os.makedirs("translated")

    # Translating Korean to English and storing in a file
    output_file_ko_en = f"translated/translated_ko_to_en_epoch_{epoch_index}.txt"
    with open(test_data_ko, "r", encoding="utf-8") as ko_file, \
         open(output_file_ko_en, "w", encoding="utf-8") as out_ko_en:
        for line in ko_file:
            translated_line = translator(line.strip())
            out_ko_en.write(translated_line + "\n")

    # Translating English to Korean and storing in a file
    output_file_en_ko = f"translated/translated_en_to_ko_epoch_{epoch_index}.txt"
    with open(test_data_en, "r", encoding="utf-8") as en_file, \
         open(output_file_en_ko, "w", encoding="utf-8") as out_en_ko:
        for line in en_file:
            translated_line = translator(line.strip())
            out_en_ko.write(translated_line + "\n")
    end_time = timer()
    print(f"Test run time = {(end_time - start_time):.3f}s")

In [ ]:
transformer_ko_to_en, optimizer_ko_to_en, loss_ko_to_en, vocab_transform_ko_to_en, text_transform_ko_to_en = init_transformer_model('ko', 'en', FILE_PATH + 'ted_train_en-ko.raw.ko', FILE_PATH + 'ted_train_en-ko.raw.en', FILE_PATH + 'ted_dev_en-ko.raw.ko', FILE_PATH + 'ted_dev_en-ko.raw.en')
transformer_en_to_ko, optimizer_en_to_ko, loss_en_to_ko, vocab_transform_en_to_ko, text_transform_en_to_ko = init_transformer_model('en', 'ko', FILE_PATH + 'ted_train_en-ko.raw.en', FILE_PATH + 'ted_train_en-ko.raw.ko', FILE_PATH + 'ted_dev_en-ko.raw.en', FILE_PATH + 'ted_dev_en-ko.raw.ko')

Initializing model for ko-to-en translation...
Source Vocabulary Size: 289862
Target Vocabulary Size: 54533


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Total number of parameters: 216930565


Initializing model for en-to-ko translation...
Source Vocabulary Size: 54533
Target Vocabulary Size: 289862


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Total number of parameters: 337654342




In [ ]:
translator = BidirectionalTranslator(transformer_ko_to_en, vocab_transform_ko_to_en, text_transform_ko_to_en, transformer_en_to_ko, vocab_transform_en_to_ko, text_transform_en_to_ko)

In [ ]:
for i in range(0, 10): # number of training epochs
  train_model(transformer_ko_to_en, optimizer_ko_to_en, loss_ko_to_en, 'ko', 'en', FILE_PATH + 'ted_train_en-ko.raw.ko', FILE_PATH + 'ted_train_en-ko.raw.en', FILE_PATH + 'ted_dev_en-ko.raw.ko', FILE_PATH + 'ted_dev_en-ko.raw.en', text_transform_ko_to_en, NUM_EPOCHS=1)
  train_model(transformer_en_to_ko, optimizer_en_to_ko, loss_en_to_ko, 'en', 'ko', FILE_PATH + 'ted_train_en-ko.raw.en', FILE_PATH + 'ted_train_en-ko.raw.ko', FILE_PATH + 'ted_dev_en-ko.raw.en', FILE_PATH + 'ted_dev_en-ko.raw.ko', text_transform_en_to_ko, NUM_EPOCHS=1)
  run_test(translator, i, FILE_PATH + 'ted_test1_en-ko.raw.ko', FILE_PATH + 'ted_test1_en-ko.raw.en')

### Download the translated data

In [31]:
!zip -r translated.zip translated

updating: translated/ (stored 0%)
updating: translated/translated_ko_to_en_epoch_0.txt (deflated 93%)
updating: translated/translated_en_to_ko_epoch_1.txt (deflated 93%)
updating: translated/translated_ko_to_en_epoch_1.txt (deflated 92%)
updating: translated/translated_en_to_ko_epoch_3.txt (deflated 89%)
updating: translated/translated_ko_to_en_epoch_2.txt (deflated 91%)
updating: translated/translated_en_to_ko_epoch_4.txt (deflated 88%)
updating: translated/translated_en_to_ko_epoch_2.txt (deflated 91%)
updating: translated/translated_ko_to_en_epoch_4.txt (deflated 89%)
updating: translated/translated_ko_to_en_epoch_3.txt (deflated 90%)
updating: translated/translated_en_to_ko_epoch_0.txt (deflated 97%)
  adding: translated/translated_en_to_ko_epoch_5.txt (deflated 88%)
  adding: translated/translated_ko_to_en_epoch_5.txt (deflated 88%)
  adding: translated/translated_en_to_ko_epoch_7.txt (deflated 87%)
  adding: translated/translated_ko_to_en_epoch_7.txt (deflated 87%)
  adding: tran

In [32]:
from google.colab import files
files.download('translated.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>